# Gradient descent using pytorch.nn

In [1]:
# Import packages
import numpy as np
import torch
from Code.gdtnn import *

ModuleNotFoundError: No module named 'Code'

In [ ]:
# Make data
n_data = 101
x = torch.linspace(-1, 1, n_data).reshape(n_data, 1)

# Target functions, from easy to hard
y1 = torch.maximum(2 * x, torch.zeros_like(x))
y2 = (x - 0.2)**3
y3 = np.sin(2 * np.pi * (x + 0.2))
y = y1 + y2 + y3

# Get cpu or gpu device for training
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
x, y = x.to(device), y.to(device)

# Make neural net with set number of neurons in hidden layer
n_hidden = 10

In [ ]:
# Train grid and non grid models on multiple random functions and compare results
n_targets = 10
n_models = 10
grid_loss_vec = torch.zeros(n_targets)
loss_mat = torch.zeros((n_targets, n_models))
for t in range(n_targets):
    print(f"target {t}")
    t_weights = torch.randn((1, 3))
    y = (t_weights @ torch.stack((y1, y2, y3), dim=1))[:, :, 0].to(device)
    model = ToyNN(n_hidden, grid=True).to(device)
    data_list, loss_vec = grad_desc(x, y, model, n_data, n_hidden, lr=1e-1, max_it=int(2e3), plot_every=2e2)
    data_list, loss_vec = grad_desc(x, y, model, n_data, n_hidden, lr=1e-2, max_it=int(3e3), plot_every=2e2)
    plot_animation(y.cpu(), data_list, showlegend=False, title='Grid network output while training')
    plot_loss(loss_vec)
    grid_loss_vec[t] = loss_vec[-1]
    print(f"grid loss {grid_loss_vec[t]}")
    for m in range(n_models):
        print(f"random model {m}")
        model = ToyNN(n_hidden, grid=False).to(device)
        data_list, loss_vec = grad_desc(x, y, model, n_data, n_hidden, lr=1e-1, max_it=int(2e3), plot_every=2e3)
        data_list, loss_vec = grad_desc(x, y, model, n_data, n_hidden, lr=1e-2, max_it=int(3e3), plot_every=3e3)
        loss_mat[t, m] = loss_vec[-1]
    print(f"random losses {loss_mat[t, :]}")
    print(f"mean random loss {torch.mean(loss_mat[t, :])}")
print(f"grid losses {grid_loss_vec}")
print(f"random average losses {torch.mean(loss_mat, dim=0)}")
print(f"average grid loss {torch.mean(grid_loss_vec)}")
print(f"average random loss {torch.mean(loss_mat)}")

In [ ]:
# Train grid model and show outputs
model = ToyNN(n_hidden, grid=True).to(device)
data_list, loss_vec = grad_desc(x, y, model, n_data, n_hidden, lr=1e-1, max_it=int(2e3), plot_every=50)
plot_animation(y.cpu(), data_list, showlegend=False, title='Network output while training')
plot_loss(loss_vec)
data_list, loss_vec = grad_desc(x, y, model, n_data, n_hidden, lr=1e-2, max_it=int(3e3), plot_every=50)
plot_animation(y.cpu(), data_list, showlegend=False, title='Network output while training')
plot_loss(loss_vec)

In [ ]:
# Train regular model and show outputs
model = ToyNN(n_hidden, grid=False).to(device)
data_list, loss_vec = grad_desc(x, y, model, n_data, n_hidden, lr=1e-1, max_it=int(2e3), plot_every=50)
go.Figure(data_list[0], dict(title="Initial network output")).show()
plot_animation(y.cpu(), data_list, showlegend=False, title='Network output while training')
plot_loss(loss_vec)
data_list, loss_vec = grad_desc(x, y, model, n_data, n_hidden, lr=1e-2, max_it=int(3e3), plot_every=50)
go.Figure(data_list[0], dict(title="Initial network output")).show()
plot_animation(y.cpu(), data_list, showlegend=False, title='Network output while training')
plot_loss(loss_vec)